## FINAL MODEL - FULL DATA RUN 

Last Execution 12:43:37 AM

Execution Time 141m 49.5s - 2 hrs 35 mins


In [ ]:
# ==================== MULTI-HEAD ANN — Integrated v2 (ringing-focused upgrades) ====================
import os, random, json, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, regularizers, losses

# -------------------- Settings --------------------
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

UNSEEN_PART = 'C2M0040120D'
DATA_CSV = r"C:\Users\pc\Desktop\Neural_Network_Models\merged_train_5_MOSFETs.csv"

BASE_DIR = "Full_Data_Run_MultiHead_ANN"
for sub in ["r2_rmse_tables","train_val_loss_curves","predicted_vs_actual","models","artifacts"]:
    os.makedirs(f"{BASE_DIR}/{sub}", exist_ok=True)

# -------------------- Targets & columns --------------------
TARGET_COLUMNS = [
    'voltage_rise_time_pulse1','voltage_rise_time_pulse2',
    'voltage_fall_time_pulse1','voltage_fall_time_pulse2',
    'current_rise_time_pulse1','current_rise_time_pulse2',
    'current_fall_time_pulse1','current_fall_time_pulse2',
    'overshoot_pulse_1','overshoot_pulse_2',
    'undershoot_pulse_1','undershoot_pulse_2',
    'ringing_frequency_MHz'
]
DROP_COLUMNS = ['DeviceID','MOSFET','Part_Number']

LOG_TARGETS = {
    'voltage_rise_time_pulse1','voltage_rise_time_pulse2',
    'current_rise_time_pulse1','current_rise_time_pulse2',
    'ringing_frequency_MHz'
}

HARD_HEADS = {
    'voltage_rise_time_pulse1','voltage_rise_time_pulse2',
    'current_rise_time_pulse1','current_rise_time_pulse2',
    'ringing_frequency_MHz'
}

# -------------------- Load & split --------------------
df = pd.read_csv(DATA_CSV)
seen_parts = [p for p in df['Part_Number'].unique().tolist() if p != UNSEEN_PART]
train_df = df[df['Part_Number'].isin(seen_parts)].copy()
test_df  = df[df['Part_Number'] == UNSEEN_PART].copy()

# -------------------- Physics features --------------------
def safe(val, eps=1e-15): return max(float(val), eps)

def compute_physics_features(row):
    # Base quantities
    Ls = float(row[['Ls4','Ls5','Ls6','Ls7','Ls8','Ls9','Ls10','Ls11']].sum())
    Coss = float(row.get("Coss", 1e-12))
    L_eq = safe(Ls)
    C_eq = safe(Coss)

    # Resonance & derived helpers
    inv_sqrt_LC = 1.0 / np.sqrt(L_eq*C_eq)
    f_res = (inv_sqrt_LC / (2*np.pi)) / 1e6           # MHz
    sqrt_LC = np.sqrt(L_eq*C_eq)

    # Rough damping proxy (no exact R_total available)
    # Use gate/network resistances as a proxy; clamp to avoid zero/negatives
    Rg = float(row.get("Rg", 0.0))
    R1 = float(row.get("R1", 0.0))
    R_proxy = safe(Rg + R1 + 1e-3)
    # Q ≈ (1/R) * sqrt(L/C)  (dimensionally consistent for proxy)
    Q_est = (np.sqrt(L_eq / C_eq)) / R_proxy

    Vbus = float(row.get("Vbus", 0.0))
    VDS_max = float(row.get("VDS_max", 0.0))
    Vth_min = float(row.get("VGS_th_min", 0.0))
    IDmax   = float(row.get("ID_max_25C", 0.0))
    Tp1 = safe(row.get("Tp1", 1e-9))

    overshoot_est  = VDS_max - Vbus
    undershoot_est = - Vth_min
    dVdt_est = overshoot_est / Tp1
    dIdt_est = IDmax / Tp1

    # Add a couple of interaction terms often useful for ringing
    v_over_L = Vbus / safe(L_eq)
    v_over_C = Vbus / safe(C_eq)

    return pd.Series([
        f_res, overshoot_est, undershoot_est, dVdt_est, dIdt_est,
        L_eq, C_eq, sqrt_LC, inv_sqrt_LC, Q_est, v_over_L, v_over_C
    ])

for d in (train_df, test_df):
    d[['f_resonance','overshoot_est','undershoot_est','dVdt_est','dIdt_est',
      'L_eq','C_eq','sqrt_LC','inv_sqrt_LC','Q_est','v_over_L','v_over_C'
     ]] = d.apply(compute_physics_features, axis=1)

physics_features = [
    'f_resonance','overshoot_est','undershoot_est','dVdt_est','dIdt_est',
    'L_eq','C_eq','sqrt_LC','inv_sqrt_LC','Q_est','v_over_L','v_over_C'
]
INPUT_COLUMNS = [c for c in df.columns if c not in TARGET_COLUMNS + DROP_COLUMNS] + physics_features

# -------------------- Scale inputs (fit on TRAIN only) --------------------
input_scaler = StandardScaler()
input_scaler.fit(train_df[INPUT_COLUMNS])

X_train_all = input_scaler.transform(train_df[INPUT_COLUMNS]).astype('float32')
X_test_all  = input_scaler.transform(test_df[INPUT_COLUMNS]).astype('float32')

# Separate physics vs main
phys_idx = [INPUT_COLUMNS.index(c) for c in physics_features]
X_train_phys = X_train_all[:, phys_idx]
X_test_phys  = X_test_all[:, phys_idx]
X_train_main = np.delete(X_train_all, phys_idx, axis=1)
X_test_main  = np.delete(X_test_all,  phys_idx, axis=1)

# -------------------- Scale outputs (per-target; log where configured) --------------------
output_scalers = {}
y_train_scaled_cols, y_test_scaled_cols = [], []

def _prep_target(col, arr):
    if col in LOG_TARGETS:
        arr = np.log1p(np.clip(arr, a_min=0, a_max=None))
    return arr

def _inv_target(col, arr1d, scaler):
    x = scaler.inverse_transform(arr1d.reshape(-1,1)).flatten()
    if col in LOG_TARGETS:
        x = np.expm1(x)
    return x

for col in TARGET_COLUMNS:
    y_tr = train_df[col].values.reshape(-1,1)
    y_te = test_df[col].values.reshape(-1,1)

    y_tr = _prep_target(col, y_tr)
    y_te = _prep_target(col, y_te)

    scaler = MinMaxScaler() if col == 'ringing_frequency_MHz' else StandardScaler()
    scaler.fit(y_tr)
    output_scalers[col] = scaler

    y_train_scaled_cols.append(scaler.transform(y_tr).flatten())
    y_test_scaled_cols.append(scaler.transform(y_te).flatten())

y_train_scaled = np.array(y_train_scaled_cols).T.astype('float32')  # [N, 13]
y_test_scaled  = np.array(y_test_scaled_cols).T.astype('float32')

# -------------------- Train/Val split --------------------
Xtr_main, Xval_main, Xtr_phys, Xval_phys, ytr, yval = train_test_split(
    X_train_main, X_train_phys, y_train_scaled,
    test_size=0.15, random_state=SEED
)

# -------------------- Model (Script #2 backbone + ringing-focused head) --------------------
def build_multihead_ann(input_dim_main, input_dim_phys, output_names,
                        l2_reg=1e-4, dropout_rate=0.2):
    inp_main = layers.Input(shape=(input_dim_main,), name="main_inputs")
    inp_phys = layers.Input(shape=(input_dim_phys,), name="physics_inputs")

    # Backbone: 128 -> 64 with BN + L2 + Dropout (from Script #2)
    x = layers.Dense(128, kernel_regularizer=regularizers.l2(l2_reg), use_bias=False)(inp_main)
    x = layers.BatchNormalization()(x); x = layers.ReLU()(x)
    x = layers.Dropout(dropout_rate)(x)

    x = layers.Dense(64, kernel_regularizer=regularizers.l2(l2_reg), use_bias=False)(x)
    x = layers.BatchNormalization()(x); x = layers.ReLU()(x)
    x = layers.Dropout(dropout_rate)(x)

    # Physics branch (wider now)
    p = layers.Dense(64, activation="relu")(inp_phys)
    p = layers.Dense(48, activation="relu")(p)

    shared = layers.Concatenate()([x, p])

    outputs = []
    for col in output_names:
        if col == 'ringing_frequency_MHz':
            # Gated physics fusion (ringing-focused)
            h = layers.Dense(96, activation="relu")(shared)
            h = layers.Dropout(dropout_rate)(h)
            # physics gate
            gate = layers.Dense(48, activation="sigmoid")(p)
            h = layers.Concatenate()([h, layers.Multiply()([p, gate])])
            h = layers.Dense(48, activation="relu")(h)
            out = layers.Dense(1, activation="linear", name=col)(h)
        else:
            h = layers.Dense(64, activation="relu")(shared)
            h = layers.Dropout(dropout_rate)(h)
            h = layers.Dense(32, activation="relu")(h)
            # physics injection for hard heads (rise family)
            if col in HARD_HEADS:
                h = layers.Concatenate()([h, p])
                h = layers.Dense(32, activation="relu")(h)
            out = layers.Dense(1, activation="linear", name=col)(h)
        outputs.append(out)

    return models.Model(inputs=[inp_main, inp_phys], outputs=outputs)

model = build_multihead_ann(
    Xtr_main.shape[1], Xtr_phys.shape[1], TARGET_COLUMNS,
    l2_reg=1e-4, dropout_rate=0.2
)

# -------------------- Losses & weights --------------------
# Use MSE for most; Huber for ringing (robust on tails)
losses_map = {col: "mse" for col in TARGET_COLUMNS}
losses_map['ringing_frequency_MHz'] = losses.Huber(delta=0.7)

# Warm-up (balanced), then fine-tune (emphasize ringing & rises)
PHASE1_WEIGHTS = {col: 1.0 for col in TARGET_COLUMNS}
PHASE2_WEIGHTS = {col: 1.0 for col in TARGET_COLUMNS}
PHASE2_WEIGHTS.update({
    'ringing_frequency_MHz': 3.0,
    'current_rise_time_pulse2': 1.8,
    'current_rise_time_pulse1': 1.5,
    'voltage_rise_time_pulse1': 1.5,
    'voltage_rise_time_pulse2': 1.5,
})

def weights_dict_to_list(d):
    return [d.get(col, 1.0) for col in TARGET_COLUMNS]

# -------------------- Compile & train (Phase 1) --------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
    loss=[losses_map[c] for c in TARGET_COLUMNS],
    loss_weights=weights_dict_to_list(PHASE1_WEIGHTS),
    metrics=["mae"]*len(TARGET_COLUMNS)
)

early1 = callbacks.EarlyStopping(monitor="val_loss", patience=12, restore_best_weights=True, min_delta=1e-4)
rlr1   = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=6, min_lr=1e-6)

hist1 = model.fit(
    [Xtr_main, Xtr_phys],
    [ytr[:, i] for i in range(len(TARGET_COLUMNS))],
    validation_data=([Xval_main, Xval_phys], [yval[:, i] for i in range(len(TARGET_COLUMNS))]),
    epochs=90, batch_size=256, callbacks=[early1, rlr1], verbose=1
)

# -------------------- Fine-tune (Phase 2) --------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
    loss=[losses_map[c] for c in TARGET_COLUMNS],
    loss_weights=weights_dict_to_list(PHASE2_WEIGHTS),
    metrics=["mae"]*len(TARGET_COLUMNS)
)

early2 = callbacks.EarlyStopping(monitor="val_loss", patience=14, restore_best_weights=True, min_delta=1e-4)
rlr2   = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=7, min_lr=1e-6)

hist2 = model.fit(
    [Xtr_main, Xtr_phys],
    [ytr[:, i] for i in range(len(TARGET_COLUMNS))],
    validation_data=([Xval_main, Xval_phys], [yval[:, i] for i in range(len(TARGET_COLUMNS))]),
    epochs=120, batch_size=256, callbacks=[early2, rlr2], verbose=1
)

model.save(f"{BASE_DIR}/models/multihead_ann_integrated_v2.h5")

# -------------------- Loss curve --------------------
plt.figure()
plt.plot(hist1.history['loss'] + hist2.history['loss'], label='Train Loss')
plt.plot(hist1.history['val_loss'] + hist2.history['val_loss'], label='Val Loss')
plt.xlabel("Epoch"); plt.ylabel("Weighted Loss")
plt.title("Train vs Validation Loss (Integrated v2)")
plt.legend(); plt.savefig(f"{BASE_DIR}/train_val_loss_curves/loss.png"); plt.close()

# -------------------- Evaluation helpers --------------------
def inverse_one(col, arr_1d):
    scaler = output_scalers[col]
    return _inv_target(col, arr_1d, scaler)

def evaluate_and_save(X_main, X_phys, y_scaled, name, positive_only=False, scatter=False):
    y_pred_scaled_list = model.predict([X_main, X_phys], verbose=0)
    rows, y_true_raw, y_pred_raw = [], [], []

    for i, col in enumerate(TARGET_COLUMNS):
        y_true = inverse_one(col, y_scaled[:, i])
        y_pred = inverse_one(col, y_pred_scaled_list[i].reshape(-1))
        y_true_raw.append(y_true); y_pred_raw.append(y_pred)
        r2   = r2_score(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        rows.append((col, r2, rmse))

    df_results = pd.DataFrame(rows, columns=["Target","R2","RMSE"]).sort_values("R2", ascending=False)
    if positive_only:
        df_results = df_results[df_results["R2"] > 0.0]

    print(f"\n{name}:\n", df_results)
    df_results.to_csv(f"{BASE_DIR}/r2_rmse_tables/{name}.csv", index=False)

    if scatter:
        n_cols = 3
        n_rows = int(np.ceil(len(TARGET_COLUMNS) / n_cols))
        plt.figure(figsize=(15, 5*n_rows))
        for i, col in enumerate(TARGET_COLUMNS):
            yt, yp = y_true_raw[i], y_pred_raw[i]
            plt.subplot(n_rows, n_cols, i+1)
            plt.scatter(yt, yp, alpha=0.45, s=10)
            lo, hi = min(yt.min(), yp.min()), max(yt.max(), yp.max())
            plt.plot([lo, hi], [lo, hi], "r--", linewidth=1)
            plt.xlabel("Actual"); plt.ylabel("Predicted"); plt.title(col)
        plt.tight_layout(); plt.savefig(f"{BASE_DIR}/predicted_vs_actual/{name}_scatter.png"); plt.close()

    return df_results

# -------------------- Run evals --------------------
evaluate_and_save(Xtr_main, Xtr_phys, ytr, "train")
evaluate_and_save(Xval_main, Xval_phys, yval, "val")
evaluate_and_save(X_train_main, X_train_phys, y_train_scaled, "test_seen", scatter=True)
evaluate_and_save(X_test_main, X_test_phys, y_test_scaled, "unseen_all", scatter=True)
evaluate_and_save(X_test_main, X_test_phys, y_test_scaled, "unseen_positive_only", positive_only=True)

# -------------------- Artifacts --------------------
with open(f"{BASE_DIR}/artifacts/input_scaler.pkl", "wb") as f:
    pickle.dump(input_scaler, f)
with open(f"{BASE_DIR}/artifacts/output_scalers.pkl", "wb") as f:
    pickle.dump(output_scalers, f)
with open(f"{BASE_DIR}/artifacts/TARGET_COLUMNS.json", "w") as f:
    json.dump(TARGET_COLUMNS, f, indent=2)
with open(f"{BASE_DIR}/artifacts/INPUT_COLUMNS.json", "w") as f:
    json.dump(INPUT_COLUMNS, f, indent=2)
with open(f"{BASE_DIR}/artifacts/PHYSICS_COLUMNS.json", "w") as f:
    json.dump(physics_features, f, indent=2)

# -------------------- Full prediction exports --------------------
def save_full_predictions(X_main, X_phys, y_scaled, df_inputs, name):
    y_pred_scaled_list = model.predict([X_main, X_phys], verbose=0)
    preds_dict, actuals_dict = {}, {}
    for i, col in enumerate(TARGET_COLUMNS):
        y_pred = inverse_one(col, y_pred_scaled_list[i].reshape(-1))
        y_true = inverse_one(col, y_scaled[:, i])
        preds_dict[f"{col}_predicted"] = y_pred
        actuals_dict[f"{col}_actual"] = y_true
    full_df = df_inputs.reset_index(drop=True).copy()
    for col in TARGET_COLUMNS:
        full_df[f"{col}_actual"] = actuals_dict[f"{col}_actual"]
        full_df[f"{col}_predicted"] = preds_dict[f"{col}_predicted"]
    out_path = f"{BASE_DIR}/artifacts/{name}_full_predictions.csv"
    full_df.to_csv(out_path, index=False)
    print(f"Saved {name} predictions → {out_path}")

save_full_predictions(X_train_main, X_train_phys, y_train_scaled,
                      train_df[INPUT_COLUMNS + ['Part_Number']], "seen")
save_full_predictions(X_test_main, X_test_phys, y_test_scaled,
                      test_df[INPUT_COLUMNS + ['Part_Number']], "unseen")


Epoch 1/90
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 35s 17ms/step - current_fall_time_pulse1_loss: 0.1743 - current_fall_time_pulse1_mae: 0.2824 - current_fall_time_pulse2_loss: 0.2061 - current_fall_time_pulse2_mae: 0.3031 - current_rise_time_pulse1_loss: 0.1574 - current_rise_time_pulse1_mae: 0.2543 - current_rise_time_pulse2_loss: 0.2539 - current_rise_time_pulse2_mae: 0.3494 - loss: 2.6276 - overshoot_pulse_1_loss: 0.3201 - overshoot_pulse_1_mae: 0.4041 - overshoot_pulse_2_loss: 0.2477 - overshoot_pulse_2_mae: 0.3594 - ringing_frequency_MHz_loss: 0.0156 - ringing_frequency_MHz_mae: 0.1037 - undershoot_pulse_1_loss: 0.2916 - undershoot_pulse_1_mae: 0.3565 - undershoot_pulse_2_loss: 0.2547 - undershoot_pulse_2_mae: 0.3479 - voltage_fall_time_pulse1_loss: 0.1468 - voltage_fall_time_pulse1_mae: 0.2388 - voltage_fall_time_pulse2_loss: 0.1297 - voltage_fall_time_pulse2_mae: 0.2243 - voltage_rise_time_pulse1_loss: 0.1224 - voltage_rise_time_pulse1_mae: 0.2182 - voltage_rise_time_pulse2_loss: 0.2948


train:
                       Target        R2          RMSE
12     ringing_frequency_MHz  0.998615  1.227932e+00
0   voltage_rise_time_pulse1  0.995878  2.598011e-10
6   current_fall_time_pulse1  0.971739  1.979292e-09
7   current_fall_time_pulse2  0.971736  1.973276e-09
2   voltage_fall_time_pulse1  0.969873  1.191931e-09
3   voltage_fall_time_pulse2  0.969685  1.194123e-09
10        undershoot_pulse_1  0.955090  2.896487e+00
11        undershoot_pulse_2  0.947827  3.123416e+00
4   current_rise_time_pulse1  0.941288  1.186998e-08
8          overshoot_pulse_1  0.938532  3.024824e+00
5   current_rise_time_pulse2  0.935089  6.389628e-09
1   voltage_rise_time_pulse2  0.930873  1.058828e-09
9          overshoot_pulse_2  0.911120  7.528735e+00

val:
                       Target        R2          RMSE
12     ringing_frequency_MHz  0.998612  1.228868e+00
0   voltage_rise_time_pulse1  0.995359  2.756488e-10
7   current_fall_time_pulse2  0.971728  1.975731e-09
6   current_fall_time_pulse1  